<a href="https://colab.research.google.com/github/fahimku2020/fahimku2020/blob/main/Good_semantic_search_wikipedia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install -U sentence-transformers
!pip install wikipedia
!pip install nltk
!pip install tqdm
!pip install logging
!pip install re
!pip install sentence-transformers

  Using cached wikipedia-1.4.0-py3-none-any.whl
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.0/96.0 kB 3.4 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
ERROR: Could not find a version that satisfies the requirement re (from versions: none)
ERROR: No matching distribution found for re


In [ ]:
import nltk
from sentence_transformers import SentenceTransformer, util
import wikipedia
import re
import logging
from tqdm import tqdm # For progress bar

# Configure logging for better error reporting
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

nltk.download('punkt')
nltk.download('punkt_tab')

def fetch_wikipedia_sentences(topics, num_sentences_per_topic=500):
    """Fetches sentences from multiple Wikipedia topics."""
    all_sentences = []
    for topic in tqdm(topics, desc="Fetching Wikipedia sentences"):
        try:
            page = wikipedia.page(topic)
            text = page.content
            sentences = nltk.sent_tokenize(text)
            cleaned_sentences = [s for s in sentences if len(re.findall(r'\d+', s)) < 5 and len(re.findall(r'[^a-zA-Z0-9\s]', s)) < 10]
            all_sentences.extend(cleaned_sentences[:num_sentences_per_topic])
        except wikipedia.exceptions.PageError:
            logging.warning(f"Wikipedia page not found for '{topic}'. Skipping.")
        except wikipedia.exceptions.DisambiguationError as e:
            logging.warning(f"Disambiguation error for '{topic}': {e.options}. Skipping.")
    return all_sentences


def build_semantic_search(sentences, model_name='all-mpnet-base-v2'):
    """Builds the semantic search model."""
    model = SentenceTransformer(model_name)
    embeddings = model.encode(sentences, show_progress_bar=True)
    return model, embeddings, sentences


def query_semantic_search(model, embeddings, sentences, query, top_k=5):
    """Performs a semantic search query."""
    query_embedding = model.encode(query)
    cosine_scores = util.cos_sim(query_embedding, embeddings)[0]
    top_indices = cosine_scores.argsort()[-top_k:]
    top_sentences = [sentences[i] for i in top_indices]
    return top_sentences


if __name__ == "__main__":
    topics = input("Enter Wikipedia topics separated by commas (e.g., 'Pakistan,India,Artificial intelligence'): ").split(',')
    topics = [t.strip() for t in topics]  #remove extra spaces

    all_sentences = fetch_wikipedia_sentences(topics)
    if not all_sentences:
        print("No sentences fetched from Wikipedia.")
        exit()

    model, embeddings, sentences = build_semantic_search(all_sentences)

    while True:
        query = input("Enter your query (or type 'exit' to quit): ")
        if query.lower() == 'exit':
            break

        results = query_semantic_search(model, embeddings, sentences, query)
        if results:
            print("\nTop", len(results), "most similar sentences:")
            for i, sentence in enumerate(results):
                print(f"{i+1}. {sentence}")
        else:
            print("No similar sentences found.")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Enter Wikipedia topics separated by commas (e.g., 'Pakistan,India,Artificial intelligence'): Pakistan


Fetching Wikipedia sentences: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Batches:   0%|          | 0/16 [00:00<?, ?it/s]

Enter your query (or type 'exit' to quit): History about country

Top 5 most similar sentences:
1. This period is marked by prolonged stagflation, political instability, corruption, misgovernment, geopolitical rivalry with India, and the clash of left wing-right wing ideologies.
2. Pakistan is the site of several ancient cultures, including the 8,500-year-old Neolithic site of Mehrgarh in Balochistan, the Indus Valley Civilisation of the Bronze Age, and the ancient Gandhara civilisation.
3. Historically, Pakistan was part of the wealthiest region in the first millennium CE, but lost ground to regions like China and Western Europe by the 18th century.
4. Pakistan's political history since independence has been characterized by periods of significant economic and military growth as well as those of political and economic instability.
5. The World Factbook.
Enter your query (or type 'exit' to quit): Climate of country

Top 5 most similar sentences:
1. It is the fifth-most populous country